In [1]:
import pandas as pd

print(pd.__version__)

0.23.4


In [2]:
#load data
dftrain=pd.read_csv('https://raw.githubusercontent.com/mulargui/Kaggle-Walmart-fcst-tensorflow/master/kaggle/train.csv')
dftest=pd.read_csv('https://raw.githubusercontent.com/mulargui/Kaggle-Walmart-fcst-tensorflow/master/kaggle/test.csv')

print(dftrain.head())
print(dftrain.shape[0])
print(dftest.head())
print(dftest.shape[0])

   Store  Dept        Date  Weekly_Sales  IsHoliday
0      1     1  2010-02-05      24924.50      False
1      1     1  2010-02-12      46039.49       True
2      1     1  2010-02-19      41595.55      False
3      1     1  2010-02-26      19403.54      False
4      1     1  2010-03-05      21827.90      False
421570
   Store  Dept        Date  IsHoliday
0      1     1  2012-11-02      False
1      1     1  2012-11-09      False
2      1     1  2012-11-16      False
3      1     1  2012-11-23       True
4      1     1  2012-11-30      False
115064


In [0]:
##########################################################
# This section generates Date features
##########################################################

In [4]:
#Add Seasonality Column based on Week of the Date
dftrain['Seasonality'] = pd.to_datetime(dftrain['Date']).apply(lambda x: x.weekofyear)
dftest['Seasonality'] = pd.to_datetime(dftest['Date']).apply(lambda x: x.weekofyear)

print(dftrain.shape[0])
print(dftest.shape[0])

421570
115064


In [5]:
#Add Year column based on Date
dftrain['Year'] = pd.to_datetime(dftrain['Date']).apply(lambda x: x.year)
dftest['Year'] = pd.to_datetime(dftest['Date']).apply(lambda x: x.year)

print(dftrain.shape[0])
print(dftest.shape[0])

421570
115064


In [0]:
##########################################################
# This section adjusts sales in christmas across 2010,2011 and 2012
##########################################################

In [7]:
#use sales in week 51 in 2010 to adjust christmas seasonality
#filter rows affected
dfchristmas = dftrain[(dftrain.Year == 2010) & (dftrain.Seasonality > 50) & (dftrain.Seasonality < 52)]

print(dfchristmas.shape[0])

2956


In [0]:
#adjust only those with all 3 weeks of sales
#dfchristmas = dfchristmas.groupby(['Store', 'Dept']).filter(lambda x: len(x) == 3 )

#print(dfchristmas.shape[0])

In [9]:
#adjust sales in weeks 51 and 52
dfgrouped = dfchristmas.groupby(['Store', 'Dept'])
for name, group in dfgrouped:
  #one day of sales
  inc_sales = group['Weekly_Sales'].sum() / 7
  dftrain.loc[(dftrain.Store == name[0]) & (dftrain.Dept == name[1]) & \
    (dftrain.Year == 2010) & (dftrain.Seasonality == 51),'Weekly_Sales'] -= (3*inc_sales)
  dftrain.loc[(dftrain.Store == name[0]) & (dftrain.Dept == name[1]) & \
    (dftrain.Year == 2010) & (dftrain.Seasonality == 52),'Weekly_Sales'] += (3*inc_sales)

print(dftrain.shape[0])

421570


In [10]:
#use sales in weeks 51 in 2011 to adjust christmas seasonality
#filter rows affected
dfchristmas = dftrain[(dftrain.Year == 2011) & (dftrain.Seasonality == 51)]

print(dfchristmas.shape[0])

3027


In [0]:
#adjust only those with all 4 weeks of sales
#dfchristmas = dfchristmas.groupby(['Store', 'Dept']).filter(lambda x: len(x) == 4 )

#print(dfchristmas.shape[0])

In [12]:
#adjust sales in weeks 49 and 52
dfgrouped = dfchristmas.groupby(['Store', 'Dept'])
for name, group in dfgrouped:
  #one day of sales
  inc_sales = group['Weekly_Sales'].sum() / 7
  dftrain.loc[(dftrain.Store == name[0]) & (dftrain.Dept == name[1]) & \
    (dftrain.Year == 2011) & (dftrain.Seasonality == 51),'Weekly_Sales'] -= (2*inc_sales)
  dftrain.loc[(dftrain.Store == name[0]) & (dftrain.Dept == name[1]) & \
    (dftrain.Year == 2011) & (dftrain.Seasonality == 52),'Weekly_Sales'] += (2*inc_sales)

print(dftrain.shape[0])

421570


In [0]:
##########################################################
# This section adds promotion features to the dataset
##########################################################

In [14]:
#load features
dfstores=pd.read_csv('https://raw.githubusercontent.com/mulargui/Kaggle-Walmart-fcst-tensorflow/master/kaggle/stores.csv')
dffeatures=pd.read_csv('https://raw.githubusercontent.com/mulargui/Kaggle-Walmart-fcst-tensorflow/master/kaggle/features.csv')

print(dfstores.head())
print(dfstores.shape[0])
print(dffeatures.head())
print(dffeatures.shape[0])

   Store Type    Size
0      1    A  151315
1      2    A  202307
2      3    B   37392
3      4    A  205863
4      5    B   34875
45
   Store        Date  Temperature  Fuel_Price  MarkDown1  MarkDown2  \
0      1  2010-02-05        42.31       2.572        NaN        NaN   
1      1  2010-02-12        38.51       2.548        NaN        NaN   
2      1  2010-02-19        39.93       2.514        NaN        NaN   
3      1  2010-02-26        46.63       2.561        NaN        NaN   
4      1  2010-03-05        46.50       2.625        NaN        NaN   

   MarkDown3  MarkDown4  MarkDown5         CPI  Unemployment  IsHoliday  
0        NaN        NaN        NaN  211.096358         8.106      False  
1        NaN        NaN        NaN  211.242170         8.106       True  
2        NaN        NaN        NaN  211.289143         8.106      False  
3        NaN        NaN        NaN  211.319643         8.106      False  
4        NaN        NaN        NaN  211.350143         8.106      Fa

In [15]:
#transform store type to dummies
dfstores = pd.concat([dfstores.drop('Type', axis=1), pd.get_dummies(dfstores['Type'], prefix='Type')], axis=1)

#normalize features
dfstores['Size']=(dfstores['Size']-dfstores['Size'].min())/(dfstores['Size'].max()-dfstores['Size'].min())

dffeatures['Temperature']=(dffeatures['Temperature']-dffeatures['Temperature'].min())/(dffeatures['Temperature'].max()-dffeatures['Temperature'].min())
dffeatures['Fuel_Price']=(dffeatures['Fuel_Price']-dffeatures['Fuel_Price'].min())/(dffeatures['Fuel_Price'].max()-dffeatures['Fuel_Price'].min())
dffeatures['MarkDown1']=(dffeatures['MarkDown1']-dffeatures['MarkDown1'].min())/(dffeatures['MarkDown1'].max()-dffeatures['MarkDown1'].min())
dffeatures['MarkDown2']=(dffeatures['MarkDown2']-dffeatures['MarkDown2'].min())/(dffeatures['MarkDown2'].max()-dffeatures['MarkDown2'].min())
dffeatures['MarkDown3']=(dffeatures['MarkDown3']-dffeatures['MarkDown3'].min())/(dffeatures['MarkDown3'].max()-dffeatures['MarkDown3'].min())
dffeatures['MarkDown4']=(dffeatures['MarkDown4']-dffeatures['MarkDown4'].min())/(dffeatures['MarkDown4'].max()-dffeatures['MarkDown4'].min())
dffeatures['MarkDown5']=(dffeatures['MarkDown5']-dffeatures['MarkDown5'].min())/(dffeatures['MarkDown5'].max()-dffeatures['MarkDown5'].min())
dffeatures['CPI']=(dffeatures['CPI']-dffeatures['CPI'].min())/(dffeatures['CPI'].max()-dffeatures['CPI'].min())
dffeatures['Unemployment']=(dffeatures['Unemployment']-dffeatures['Unemployment'].min())/(dffeatures['Unemployment'].max()-dffeatures['Unemployment'].min())

print(dfstores.shape[0])
print(dffeatures.shape[0])

45
8190


In [0]:
#remove the features with NaNs
dffeatures.drop("MarkDown1", axis=1, inplace=True) 
dffeatures.drop("MarkDown2", axis=1, inplace=True) 
dffeatures.drop("MarkDown3", axis=1, inplace=True) 
dffeatures.drop("MarkDown4", axis=1, inplace=True) 
dffeatures.drop("MarkDown5", axis=1, inplace=True) 
dffeatures.drop("CPI", axis=1, inplace=True) 
dffeatures.drop("Unemployment", axis=1, inplace=True) 

In [17]:
#merge the features in the dataframes
dftrain = pd.merge(dftrain, dfstores, on='Store', how='left')
dftrain = pd.merge(dftrain, dffeatures, on=['Store', 'Date'], how='left')
dftest = pd.merge(dftest, dfstores, on='Store', how='left')
dftest = pd.merge(dftest, dffeatures, on=['Store', 'Date'], how='left')

print(dftrain.shape[0])
print(dftest.shape[0])

421570
115064


In [0]:
##########################################################
# This section normalizes and cleans up
##########################################################

In [19]:
#validate that IsHoliday is consistent in the dataframes
for index, row in dftrain.iterrows():
  if row['IsHoliday_x'] != row['IsHoliday_y']:
    print(row)
for index, row in dftest.iterrows():
  if row['IsHoliday_x'] != row['IsHoliday_y']:
    print(row)
    
#IsHoliday is duplicated, remove one
dftrain.drop("IsHoliday_y", axis=1, inplace=True) 
dftest.drop("IsHoliday_y", axis=1, inplace=True) 

#transform IsHoliday_x type to dummies
dftrain = pd.concat([dftrain.drop('IsHoliday_x', axis=1), pd.get_dummies(dftrain['IsHoliday_x'], prefix='IsHoliday')], axis=1)
dftest = pd.concat([dftest.drop('IsHoliday_x', axis=1), pd.get_dummies(dftest['IsHoliday_x'], prefix='IsHoliday')], axis=1)

print(dftrain.shape[0])
print(dftest.shape[0])

421570
115064


In [0]:
#normalize Stores (but keep the old ones for submission)
dftrain['Store2']=(dftrain['Store']-dftrain['Store'].min())/(dftrain['Store'].max()-dftrain['Store'].min())
dftrain['Dept2']=(dftrain['Dept']-dftrain['Dept'].min())/(dftrain['Dept'].max()-dftrain['Dept'].min())

dftest['Store2']=(dftest['Store']-dftest['Store'].min())/(dftest['Store'].max()-dftest['Store'].min())
dftest['Dept2']=(dftest['Dept']-dftest['Dept'].min())/(dftest['Dept'].max()-dftest['Dept'].min())

In [0]:
#normalize Year
dftrain['Year']=(dftrain['Year']-2010)/(2013-2010)
dftrain['Seasonality']=(dftrain['Seasonality']-1)/(52-1)

dftest['Year']=(dftest['Year']-2010)/(2013-2010)
dftest['Seasonality']=(dftest['Seasonality']-1)/(52-1)

In [22]:
#move the label column (Weekly_Sales) to the end 
dftrain = dftrain.drop(columns='Weekly_Sales').assign(Weekly_Sales=dftrain['Weekly_Sales'])

print(dftrain.head())
print(dftrain.shape[0])
print(dftest.head())
print(dftest.shape[0])

   Store  Dept        Date  Seasonality  Year      Size  Type_A  Type_B  \
0      1     1  2010-02-05     0.078431   0.0  0.630267       1       0   
1      1     1  2010-02-12     0.098039   0.0  0.630267       1       0   
2      1     1  2010-02-19     0.117647   0.0  0.630267       1       0   
3      1     1  2010-02-26     0.137255   0.0  0.630267       1       0   
4      1     1  2010-03-05     0.156863   0.0  0.630267       1       0   

   Type_C  Temperature  Fuel_Price  IsHoliday_False  IsHoliday_True  Store2  \
0       0     0.454046    0.050100                1               0     0.0   
1       0     0.419260    0.038076                0               1     0.0   
2       0     0.432259    0.021042                1               0     0.0   
3       0     0.493592    0.044589                1               0     0.0   
4       0     0.492402    0.076653                1               0     0.0   

   Dept2  Weekly_Sales  
0    0.0      24924.50  
1    0.0      46039.49  

In [0]:
##########################################################
# This section saves the data
##########################################################

In [24]:
#Save the dataframes in csv files in Google Drive
from google.colab import drive
drive.mount('/content/drive/')
dftrain.to_csv('/content/drive/My Drive/train.csv', na_rep='NA', index=False)
dftest.to_csv('/content/drive/My Drive/test.csv', na_rep='NA',index=False)
!ls -la "/content/drive/My Drive/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/
total 87079
drwx------ 2 root root     4096 Mar 27 17:01 'Colab Notebooks'
-rw------- 1 root root  1355555 Apr 14 23:19  prediction.csv
-rw------- 1 root root  2915523 Apr 14 23:21  Submission.csv
-rw------- 1 root root 16827634 Apr 17 03:06  test.csv
-rw------- 1 root root 68065003 Apr 17 03:05  train.csv


In [0]:
#Save the dataframes in csv files (locally in the VM)
#dftrain.to_csv('train.csv', na_rep='NA', index=False)
#dftest.to_csv('test.csv', na_rep='NA', index=False)

#Download the csv files
#from google.colab import files
#files.download('train.csv')
#files.download('test.csv')

#that's it for now!